# The Career Decisions of Young Men

This notebook processes and explores the data for the seminal work by Michael Keane and Kenneth Wolpin studying the career decisions of young men.

> Michael P. Keane, Kenneth I. Wolpin (1997). [The Career Decisions of Young Men](http://www.journals.uchicago.edu/doi/10.1086/262080). *Journal of Political Economy*, 105(3): 473-522.

The original file 

## Preparations

We first peform some basic preparations on the original dataset that eases further processing.

In [71]:
import pandas as pd
import numpy as np

columns = ['Identifier', 'Age', 'Schooling', 'Choice', 'Wage']
dtype = {'Identifier': np.int, 'Age': np.int,  'Schooling': np.int,  'Choice': 'category'}

df = pd.DataFrame(np.genfromtxt('KW_97.txt'), columns=columns).astype(dtype)
df.set_index(['Identifier', 'Age'], inplace=True, drop=False)
df["Choice"].cat.categories = ['Schooling', 'Home', 'White', 'Blue', 'Military']

## Basic Descriptives

We start by reproducing some basic descriptive statistics from the paper.

### Choice Probabilities

We reproduce the choice probabilities reported in Tabel 1 of the orignal paper.

In [72]:
# Please reproduce Table 1

### Average Real Wages

We reproduce the average real wages by occpation reported in Table 4 of the original paper.

In [73]:
# Please reproduce Table 2

## State Variables

We extend the original data and derive the full set of state variables.


* Blue-collar work experience, linear and squared

* White-collar work experience, linear and squared

* Military work experience, linear

In [78]:
# Please compute the additional state variables

## Further Processing

We simply store a DataFrame and a simple text file to ease further processig. These checked into version control so they can be directly downloaded by interested researchers.

In [77]:
fname = 'career_decisions_data'
df.to_pickle(fname + '.pkl')
df.to_csv(fname + '.csv', index=False)